## Qiskit textbook, volume 1
### O funcionamento de computadores clássicos

In [ ]:
# Célula de importações
import matplotlib
from qiskit import QuantumCircuit, assemble, Aer
from qiskit.visualization import plot_histogram, plot_bloch_vector#, visualize_transition
from math import sqrt, pi, pow, cos, sin, e
from numpy import arccos

# OBS: Após baixar, descomentar o "visualize_transition", acontece que ele deixa o arquivo .ipynb incrivelmente pesado
    

### Dividindo informações em bits

In [ ]:
#Primeiro exercício, retorna o binário de um número
def exercicio_01(n):
    return f'{n:04b}'
binsd = exercicio_01(72)
print(f"72 em binário seria {binsd}")

# Segundo exercício, apenas responde uma pergunta
def exercício_02(a):
    pergunta = "Se você tem um número N de bits, em quantos estados diferentes podemos representa-los?"
    print(pergunta)
    print(f"Resposta: {a}")
exercício_02("Levando em conta que cada bit contém 2 estados de informação, 2^n")

### Criando um circuito quântico

In [ ]:
qc_out = QuantumCircuit(8) # Cria um circuito com 8 bits quânticos
qc_out.measure_all() # Faz a medição do circuito, criando 8 bits de saída

def simular_circuito(qc):
    simulador = Aer.get_backend('aer_simulator') # Backend de simulação
    resultado = simulador.run(qc).result()
    counter = resultado.get_counts()
    return counter

plot_histogram(simular_circuito(qc_out))

### Introdução a portas lógicas
#### NOT gate

In [ ]:
# A porta NOT é bastante simples, ela simplesmente inverte o valor de um bit NOT(0) = 1 && NOT(1) = 0
# Para Qubits, esta porta é conhecida como " x ", ou " pauli-x "

In [ ]:
qc_encode = QuantumCircuit(8) # Cria um circuito de 8 bits
qc_encode.x(7) # Utiliza uma porta x no sétimo qubit do circuito (ou o oitavo, já que começamos a contar do 0)
qc_encode.draw(output="mpl") # Desenha o circuito na tela

In [ ]:
qc_encode.measure_all() # Realiza a medição de todos os qubits, da mesma maneira que fizemos no circuito anterior
qc_encode.draw(output="mpl") # Desenha o circuito na tela

In [ ]:
plot_histogram(simular_circuito(qc_encode))

Um detalhe importante de ser mencionado é o fato de que as strings de binário do qiskit devem ser lidas da direita para a esquerda, que nem na escrita hebraica. Então ao alterar apenas o sétimo bit da string, teremos "10000000" que pode ser convertido para 128, ou 2⁷, como visto anteriormente

In [ ]:
# Exercício, codificando um número aleatório com um circuito, utilizando o mesmo número do exercício 1, 72
def exercicio_03():
    qc = QuantumCircuit(7)
    qc.x(6)
    qc.x(3)
    qc.measure_all()
    y = simular_circuito(qc)
    return y
plot_histogram(exercicio_03()) 
# Um detalhe, ao usar a função "plot_histogram" dentro de outra função eu recebo um erro do matplotlib, então por isso que estou chamando essa função sempre sozinha   

### Adicionando binários
No início deste capítulo, a documentação ensina a somar binários na mão, o que é uma ação simples porém trabalhosa, perfeita para ser automatizada com um circuito

 Como sempre, esse circuito irá codificar um input X e entregar um output Y equivalente a soma que queremos<br>
 O primeiro passo para fazer um circuito de adição, é verificarmos se dois bits são iguais ou diferentes<br>
 Fazemos isso com uma XOR gate, que basicamente age deste jeito: <br>
 <br>
  XOR(0, 0) = 0 <br>
  XOR(0, 1) = 1 <br>
  XOR(1, 0) = 1 <br>
  XOR(1, 1) = 0 <br>
  <br>
A porta quântica responsável por este trabalho é a controlled-NOT (CNOT) gate (que eu fiquei devendo no mapa mental, perdão por isso)

In [ ]:
qc_cnot = QuantumCircuit(2) # Cria um circuito com 2 qubits
qc_cnot.cx(0,1) # Esta função é aplicada a 2 qubits, um de controle o outro sendo o alvo
qc_cnot.draw(output="mpl")

A função é aplicada com os 2 qubits de input, e seu output é registrado no qubit <i>Target</i> ou <i>Alvo</i> (que é o com o círculo marcado pelo X) 

In [ ]:
qc_ha = QuantumCircuit(4,2) # Cria um circuito quântico com 4 qubits e 2 bits clássicos
qc_ha.x(0) 
qc_ha.x(1) 
qc_ha.barrier()
# use cnots to write the XOR of the inputs on qubit 2
qc_ha.cx(0,2)
qc_ha.cx(1,2)
qc_ha.barrier()
# extract outputsoutput="mpl"
qc_ha.measure(2,0) # extract XOR value
qc_ha.measure(3,1)

qc_ha.draw(output="mpl")

#Neste circuito, já temos metade do circuito que queremos para somar dois binários
#
# Se q0 == 1 && q1 == 1: q2 = 1 && q3 = 0; (q2,q3) = (1, 0) 
# Agora, precisamos aplicar uma Toffoli gate, que é equivalente a AND gate
#
# Toffoli(1 , 1, 1) => (1, 1, 0)
# Toffoli(1 , 1, 0) => (1, 1, 1)
# inverte o valor do qubit alvo caso os dois qubits-controle tiverem valor 1 

In [ ]:
qc_ha = QuantumCircuit(4,2) # Cria um circuito quântico com 4 qubits e 2 bits clássicos
qc_ha.x(0) 
qc_ha.x(1) 
qc_ha.barrier()
# use cnots to write the XOR of the inputs on qubit 2
qc_ha.cx(0,2)
qc_ha.cx(1,2)
qc_ha.ccx(0, 1, 3)
qc_ha.barrier()
# extract outputsoutput="mpl"
qc_ha.measure(2,0) # extract XOR value
qc_ha.measure(3,1)

qc_ha.draw(output="mpl")

In [ ]:
simulador = Aer.get_backend('aer_simulator') # Backend de simulação
qobj = assemble(qc_ha)
counts = simulador.run(qobj).result().get_counts()
plot_histogram(counts)

A partir de agora eu vou escrever um pouco menos, visto que o resto do capítulo é em sua grande parte composto por explicações teóricas. <br> Pelo tempo acho que não vai ser possível aplicar nesta atividade, mas eu vi pela internet que é possível escrever em LaTeX no jupyter, vou ver se aprendo e aplico nas próximas atividades <br><br>
Por enquanto vou fazer anotações em markdown 

### State vectors
Eu achei os vetores de estado muito parecidos com os dados comumente denominados como "y" em uma rede neural após aplicado o one-hot encoding, como se o mesmo fosse um vetor de estado sempre clássico. 
<br>
<img src="sources/state_vectors.png" alt="State Vector"/>
<br>
Um vetor de estado é uma matriz a(n,1) onde são alocadas as probabilidades de encontrarmos X elemento em Y estado, como no exemplo acima, onde é de certo que encontraremos um carro na posição 4

### Notação de qubits
Bits comuns podem ser representados de um jeito bem simples, apenas definindo-os com os valores de 0 ou 1
    <br><center>c = 1 || c = 0 </center><br>
Já para os qubits, apenas teremos valores certos de 0 ou 1 após a medição de seu estado, antes disso o seu estado pode ser apenas definido por maneiras probabilísticas. Para isto, nós utilizamos os vetores de estado e uma notação conhecida como "<i>bra-ket notation</i>" criada pelo físico teórico Paul Dirac.
<br>
<img src="sources/bra_ket.png" alt="State Vector"/>
<center> 
    Exemplo da <i>ket notation</i> utilizada para representar os estados 0 e 1 de um qubit, respectivamente.
    <br> (Isso é basicamente o que foi visto acima, porém refinado para qubits) <br>
</center>
Algo muito especial dessa notação, é o fato de que podemos representar qubits em estados de superposição utilizando ela, segue um exemplo:
<img src="sources/superposition_ket.png" alt="State Vector"/>
<br>
<center> Para melhor entender essa notação, eu sinto que é necessário o desenvolvimento da expressão acima.</center>
    <br>
<img src="sources/kQ0et.png" alt="State Vector"/>
    <br>
E com esta ferramenta e algumas outras, podemos compreender e manipular os estados de um qubit ao favor do nosso objetivo.
<br>
<br>
Não é citado na documentação, mas vale salientar que a soma das amplitudes elevadas ao quadrado deve ser igual a 1
  


### Implementação dos conhecimentos recém adquiridos

In [ ]:
def exercicio_04(estado):
    qc = QuantumCircuit(1) # Cria circuito com 1 qubit
    # Inicialmente, os qubits sempre se econtram no estado |0> 
    qc.initialize(estado, 0) # Define o primeiro (0st) qubit como o estado inicial anteriormente definido
    qc.save_statevector() # Fala para o simulador salvar o vetor de estado que alteramos
    qobj = assemble(qc) # Monta o objeto deste específico circuito
    return qobj, qc

def simular_exercicio_04(*estado): # Simulação para o qubit [0, 1]
    if len(estado) > 0:
        estado_inicial = estado[0]
    else:
        estado_inicial = [0, 1] #exemplo do livro-texto
        
    simulador = Aer.get_backend('aer_simulator') 
    qc, qc_o = exercicio_04(estado_inicial)
    resultado = simulador.run(qc).result()
    vetor_estado_output = resultado.get_statevector()
    print("Lembre-se que no python 'j' é utilizado como o número imaginário 'i' \n")
    
    if str(vetor_estado_output[1])[0] == "(": 
        p2 = str(vetor_estado_output[1])[1]
    else: 
        p2 = str(vetor_estado_output[1])[0]
        
    if str(vetor_estado_output[0])[0] == "(": 
        p1 = str(vetor_estado_output[0])[1]
    else: 
        p1 = str(vetor_estado_output[0])[0]
        
    print(f"""
        Estados medidos:

        | {vetor_estado_output[0]} -> {p1} (ignorar caso L>1)
        | {vetor_estado_output[1]} -> {p2} (ignorar caso L>1)
    """)
    
    qc_o.measure_all()
    print(f"\n{qc_o}")
    qobj = assemble(qc_o)
    result = simulador.run(qobj).result()
    counts = result.get_counts()
    return counts
    
    

In [ ]:
x = simular_exercicio_04([0, 1])
y = simular_exercicio_04([1, 0])
z = simular_exercicio_04([1/sqrt(2), 1/sqrt(2)])

In [ ]:
plot_histogram(x)

In [ ]:
plot_histogram(y)

In [ ]:
plot_histogram(z)

### Regras de medição
Como citado anteriormente, a soma do módulos das amplitures ao quadrado devem ser iguais a 1, podemos ver isso representando matematicamente a seguir:
<img src="sources/regra_medicao.png">
Aqui vemos o primeiro uso de um "bra" que é similar ao "ket" só que ao invés de um vetor-coluna temos um vetor-linha. juntos, eles fazem uma "bra-ket" notation.

In [ ]:
def exercicio_05():
    initial_state = [1/sqrt(2), 0.+1.j/sqrt(2)]
    x = simular_exercicio_04([1/sqrt(2), 0.+1.j/sqrt(2)])
    return x
x = exercicio_05()
plot_histogram(x)

In [ ]:
# Exercicio 6
a = sqrt(1/3)
b = sqrt(1 - 1/3)

state_vector = [a, b]

a = simular_exercicio_04(state_vector)
plot_histogram(a)

In [ ]:
# Exercicio 7
a = sqrt(1/3)
b = sqrt(2/3)

state_vector = [a, b]

a = simular_exercicio_04(state_vector)
plot_histogram(a)

### A esfera de Bloch
A partir de agora, a matemática começou a ficar relativamente mais complexa, então eu vou precisar me esforçar mais um pouco para absorver todo o capítulo. <br>
Com um set de operações, podemos obter uma nova representação geral do estado de um qubit, desta vez uma representação que pode ser representada visualmente.
<img src="sources/bloch_general_representation.png"><br>
Não entendi a matemática por trás desta fórmula de primeira, porém com a ajuda de um <a href="https://www.youtube.com/watch?v=a-dIl1Y1aTs&t=9s">vídeo</a> eu consegui repetir o processo apresentado na documentação no papel.<br><br>
Eu achei a matemática por trás da esfera de block relativamente complicada, mas eu compreendi seu funcionamento e agora vou partir para alguns exercícios.<br> <br>
Um detalhe importante não citado no livrotexto é que diferentemente da função lá apresentada, a ordem dos parâmetros no qiskit.visualization.plot_bloch_vector() deve ser [Raio, Theta, Phi] e não [Theta, Phi, Raio]




In [ ]:
def exercicio_08():
    estado_0 = [1, 0, 0]
    estado_1 = [1, pi, 0]
    estado_2 = [1, pi/2, pi/2] # teoricamente eu penso que está certo, 50% 50%
    return [estado_0, estado_1, estado_2]

plot_bloch_vector(exercicio_08()[0], coord_type='spherical', title="Representação 1") 
plot_bloch_vector(exercicio_08()[1], coord_type='spherical', title="Representação 2") 
plot_bloch_vector(exercicio_08()[2], coord_type='spherical', title="Representação 3") 

### Portas quânticas
Nesse capítulo serão apresentadas as portas: <br>
- Pauli-X
- Pauli-Y
- Pauli-Z
- Hadamard Gate
- Porta I
- Porta S
- Porta T
- Porta U

### Portas de Pauli
#### Pauli X
A porta X pode é representada pela seguinte matriz: 
<br>
<img src="sources/pauli_x_matrix.png">
<br>
A porta X inverte o sinal do qubit alvo. Podemos utilizar ela com o código abaixo

In [ ]:
qc = QuantumCircuit(1)
qc.x(0)
qc.draw(output="mpl")

### Visualização da operação realizada pela porta X

In [ ]:
visualize_transition(qc)

### Portas Y e Z
As portas Y e Z, similarmente a porta X, realizam rotações de valor pi na esfera de bloch, cada porta realizando a rotação no eixo que está em seu nome.
<br>
<img src="sources/portas_y_z.png">

In [ ]:
qc_y = QuantumCircuit(1)
qc_y.y(0)
qc_y.draw(output="mpl")

### Visualização da operação realizada pela porta Y

In [ ]:
visualize_transition(qc_y)

### Visualização do circuito e da operação realizada pela porta Z
OBS: A porta Z realiza uma movimentação em seu eixo, logo foi necessário a aplicação de uma porta Hadamar para deixar o vetor em uma posição horizontal, e assim observarmos a movimentação realizada pela porta |

In [ ]:
qc_z = QuantumCircuit(1)
qc_z.h(0)
qc_z.z(0)
qc_z.draw(output="mpl")

In [ ]:
visualize_transition(qc_z)

### Eigenvalues
Antes de prosseguir com os exercícios desta parte do capítulo, eu precisei estudar os conceitos de "Eigenvalues" e "Eigenstates", ambos são conceitos essenciais para a mecânica quântica, como podemos ver no terceiro postulado da mecânica quântica, que diz: 
<br><br>
<center> "O único resultado possível em uma medida de uma quantidade física A é um dos
autovalores (eigenvalues) do observável correspondente A" </center>
<br><br>
Sim, mas o que isso significa? Para entender o postulado acima, precisamos entender primeiramente o conceito de operador na mecânica quântica. <br>
Um operador A é um objeto matemático que exerce influência sob um estado Phi e entrega um resultado Phi'
<br>
<img src="sources/operadores_1.png">
<br>
Engenvalues e eigenvectors agem da seguinte maneira:<br>
Temos um operador "A", que age em um estado especial "Psi" de tal maneira que a única mudança realizada pelo operador é igual a uma multiplicação escalar por uma constante, que geralmente é um número complexo.
<br>
<img src="sources/eigenvalue2.png">
<br>
Quebrando o que está escrito acima nós obtemos: <br>

- |λ> é chamado de <i>eigenstate ou eigenket ou eigenvector</i> do operador A
- λ é chamado de <i>eigenvalue</i>
- O conjunto de eigenvalues {λ} é chamado de espectro o operador A 
<br><br>
Eigenstates possuem diversas propriedades, merecendo um estudo apropriado na mecânica quântica, mas por agora tal estudo não será necessário nem introduzido pelo livrotexto. <br><br>
<img src="sources/exercicio_09.png">
<br><br>
<center> Estou tendo certa dificuldade em entender o exercício 09, vou pular ele por agora </center>

### Porta hadamard
A porta hadamard é uma das principais portas utilizadas na computação quântica, pois ela permite a criação de estados de superposição. Na esfera de Bloch ela posiciona o vetor entre os polos da esfera.
<br>
<img src="sources/hadamard_gate.png">
<br>


In [ ]:
# Aplicação de uma hadamard gate
qc_h = QuantumCircuit(1)
qc_h.h(0)
qc_h.draw(output="mpl")

In [ ]:
visualize_transition(qc_h)

### Exercício 10, demonstração visual na esfera de bloch

In [ ]:
# Exercício 10
qc_x = QuantumCircuit(1)
qc_x.x(0)
visualize_transition(qc_x)

In [ ]:
qc_hzh = QuantumCircuit(1)
qc_hzh.h(0)
qc_hzh.z(0)
qc_hzh.h(0)
visualize_transition(qc_hzh)

### Exercício 10, demonstração matemática
<img src="sources/prova_matematica_10.png">

### Porta P
A porta P realiza uma rotação no eixo Z de acordo com um parâmetro "ϕ".
<br>
<img src="sources/p_gate_matrix.png">
<br>
Abaixo, podemos ver um exemplo da aplicação da porta P

In [ ]:
qc = QuantumCircuit(1, 1)
qc.h(0) # Posiciona o vetor de bloch entre os polos |0>, |1> 
qc.p(pi, 0) # O vetor de bloch irá realizar uma rotação de valor Pi na esfera
qc.h(0) # Reposiciona o vetor de bloch em um dos plos da esfera, entregando um resultado semelhante a porta X
qc.measure(0, 0)
simulador = Aer.get_backend('aer_simulator') # Backend de simulação
qobj = assemble(qc)
counts = simulador.run(qobj).result().get_counts()
plot_histogram(counts)

In [ ]:
qc.draw(output="mpl")

### Portas I, S e T
<br>

#### Porta I
<br>
A porta I é uma matriz identidade, ela não faz nada quando utilizada como operador, logo é apenas utilizada matematicamente e como uma operação nula.
<br><br>
Exercício 11: <br>
<img src="sources/exercicio11.png">
<br>

#### Porta S 
<br>
A porta S age como uma porta P tendo como seu parâmetro "ϕ" o valor "π/2", logo ao aplica-la como um operador o vetor de bloch realiza 1/4 de volta ao redor da esfera. A porta S tem uma variação "S†" que realiza a mesma coisa, porém com sentido contrário já que seu equivalente é uma porta P(−π/2). <br>
A porta S também é chamada de "√Z", já que quando aplicada duas vezes tem a mesma função que uma porta Z. <br>
Abaixo podemos ver essa porta sendo aplicada
 
 

In [ ]:
qc = QuantumCircuit(1)
qc.h(0)
qc.s(0)
qc.sdg(0)
qc.draw(output="mpl")

In [ ]:
visualize_transition(qc)

### Porta T
A porta T tem seu funcinamento identico ao de uma porta P com seu parâmetro ϕ de valor π/4, logo também sendo chamada de "4√Z" <br>
<img src="sources/t_gate_matrix.png">
 <br>
 Também existe a porta T†, ou "T-dagger", que executa a mesma rotação da porta T porém em sentido oposto.

In [ ]:
qc = QuantumCircuit(1)
qc.h(0)
qc.t(0)
qc.tdg(0)
visualize_transition(qc)

### Porta U
A porta U é uma porta mais geral que pode executar as funções de todas as outras portas que existem nesse capítulo, mas seu uso não é tão comum pela dificuldade de leitura que ela representa
<br>
<img src="sources/u_gate_matrix.png">

In [ ]:
qc = QuantumCircuit(1, 1)
qc.u(pi/2, 0, pi, 0)
qc.measure(0, 0)
qobj = assemble(qc)
counts = simulador.run(qobj).result().get_counts()
qc.draw(output="mpl")

In [ ]:

plot_histogram(counts)

#### Aqui se finaliza a primeira parte do livrotexto, criarei outro notebook para o capítulo 2